In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, pandas as pd

# 설정
MAX_POSTS = 30
BASE_SEARCH_URL = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword="

# 드라이버 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 사용자 입력
category = input("구분 업종: ")
keyword = input("어떤 키워드로 검색?: ")
search_url = BASE_SEARCH_URL + keyword
driver.get(search_url)
time.sleep(3)

# 게시물 정보 수집
results = []

items = driver.find_elements(By.CSS_SELECTOR, "div.list_search_post > div.item")[:MAX_POSTS]

for item in items:
    try:
        title = item.find_element(By.CSS_SELECTOR, "div.desc strong.title_post span.title").text
    except:
        title = ""

    try:
        description = item.find_element(By.CSS_SELECTOR, "div.desc a.text").text
    except:
        description = ""

    try:
        date = item.find_element(By.CSS_SELECTOR, "div.writer_info span.date").text
    except:
        date = ""

    try:
        thumbnail = item.find_element(By.CSS_SELECTOR, "div.thumbnail_area img.img_post").get_attribute("src")
    except:
        thumbnail = ""

    results.append({
        "채널/기법": "n_blog",
        "구분업종": category,
        "검색 키워드": keyword,
        "제목": title,
        "미리보기": description,
        "작성일": date,
        "썸네일": thumbnail
    })

In [6]:
from datetime import datetime

# 현재 시간 문자열 (2025-07-29_09-30-15)
now_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 파일명 생성
filename = f"n_blog_crawling_result_{now_str}.csv"

# 저장
df = pd.DataFrame(results)
df.to_csv(filename, index=False, encoding="utf-8-sig")
print(f"CSV 저장 완료: {filename}")

CSV 저장 완료: n_blog_crawling_result_2025-07-29_16-52-00.csv
